#**Install Dependencies**

In [1]:
!pip install -q transformers bitsandbytes peft trl accelerate xformers wandb datasets einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.5/100.5 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━

#**Higgingface login**

In [20]:
from huggingface_hub import notebook_login
notebook_login()

#**Importing Dependencies**

In [39]:
from huggingface_hub import notebook_login
from datasets import load_dataset, Dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer, TrainingArguments, Trainer,
                          DataCollatorForLanguageModeling, BitsAndBytesConfig,
                          TextStreamer, GenerationConfig)
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig
import torch, os, wandb
wandb.login(key = "881d9a8fc90c336254f9e2e49910716a8509e9d6")
new_model = "/content/drive/MyDrive/phi-1.5b-bf16-finetuned-medicine-conversational"
base_model = "microsoft/phi-1_5"

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [4]:
#loading the model and tokenizer
bitsandbytes= BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    base_model, device_map={"":0},
    quantization_config= bitsandbytes, trust_remote_code= True
)
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token


config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.8k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**wandb login**

In [7]:
run = wandb.init(project='Fine tuning microsoft phi -1.5', job_type="training", anonymous="allow")

#**Lora Hyperparameter**
####**r (Lora Rank)**
Rank of the update matrices. If the value of rank small results in smaller update matrices with less trainable parameters. Default value of r is 8.
####**lora_alpha (Lora scaling factor)**
Scale the weights matrix consistently with pretarained weights. Default value of lora_alpha is 8.
Note: Keeping the lora_alpha two time as compare to "r" cause best results.
####**target_modules (attention blocks or layers)**
Layers or blocks added in LORA update metrices. Default value is None.
####**lora_dropout (Dropout factor)**
Dropout the selected probability of neurons in LORA weights update. It reduce the overfitting. Default value is 0.
####**bias_type**
It specified whether update bias parameter or not.
 * None
 * all
 * lora_only

####**task_type**
It specify the type for which model is finetune. These are the following types.
 * CAUSAL_LLM
 * FEATURE EXTRACTION
 * QUESTION_ANS
 * SEQ_2_SEQ_LM
 * SEQ_CLS
 * TOKEN_CLS


In [8]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["fc1", "fc2","Wqkv", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 12,582,912 || all params: 1,430,853,632 || trainable%: 0.8793989628702986


#**Download Dataset**

In [9]:
data = load_dataset("MattBastar/Medicine_Details", "main", split="train")
data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['Medicine Name', 'Composition', 'Uses', 'Side_effects', 'Image URL', 'Manufacturer', 'Excellent Review %', 'Average Review %', 'Poor Review %'],
    num_rows: 11825
})

#**Prepare Prompt**

In [10]:
def make_prompt()-> str:
    """
    make_prompt take code as input and retirn the prompt with the given
    pargraph.

    Parameters
    ----------
    None
    Return
    ------
    prompt: str
        prompt to find the nouns from given paragraph.
    """

    file_path = "/content/question_prompt.txt"
    with open(file_path, "r", encoding = "utf8") as file:
        prompt = file.read()
    return prompt

#**Prepare Dataset**

In [11]:
def format_input(example: dict):
  # example["text"] = f'''<HUMAN>:Tell me about {example["Medicine Name"]}? | What is the purpose of {example["Medicine Name"]}? | What is the use of {example["Medicine Name"]}? | What is the composition of {example["Medicine Name"]}? | What are the side effects of {example["Medicine Name"]}? | What are the reviews of {example["Medicine Name"]}? | Give me the image of {example["Medicine Name"]}?
  # <ASSISTANT>: Composition: {example["Composition"]} \n Uses: {example["Uses"]} \n Side_effects: {example["Side_effects"]} \n Uses: {example["Uses"]} \n Uses: {example["Uses"]} \n Uses: {example["Uses"]}
  # '''
  prompt = make_prompt()
  prompt = prompt.format(MEDICINE = example["Medicine Name"],
                         USES = example["Uses"],
                         EFFECTS = example["Side_effects"])
  example["text"] = prompt
  return example

In [12]:
data = data.map(format_input)

Map:   0%|          | 0/11825 [00:00<?, ? examples/s]

In [13]:
data[0]

{'Medicine Name': 'Avastin 400mg Injection',
 'Composition': 'Bevacizumab (400mg)',
 'Uses': ' Cancer of colon and rectum Non-small cell lung cancer Kidney cancer Brain tumor Ovarian cancer Cervical cancer',
 'Side_effects': 'Rectal bleeding Taste change Headache Nosebleeds Back pain Dry skin High blood pressure Protein in urine Inflammation of the nose',
 'Image URL': 'https://onemg.gumlet.io/l_watermark_346,w_480,h_480/a_ignore,w_480,h_480,c_fit,q_auto,f_auto/f5a26c491e4d48199ab116a69a969be3.jpg',
 'Manufacturer': 'Roche Products India Pvt Ltd',
 'Excellent Review %': 22,
 'Average Review %': 56,
 'Poor Review %': 22,
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\\n\\n\n### Instruction:\\n\nUser Input: What are the uses of Avastin 400mg Injection? \\n\nUser Input: Avastin 400mg Injection?\\n\nUser Input: What are the side effects of Avastin 400mg Injection? \\n\\n\n### Response:\\n \nUses: Avastin 400mg Injection i

#**Tokenzing the dataset**
 * Tokenize the text field in dataset.

In [14]:
def tok(sample):
    model_inps =  tokenizer(sample["text"], padding=True)
    return model_inps
tokenized_training_data = data.map(tok, batched=True)
tokenized_training_data

Map:   0%|          | 0/11825 [00:00<?, ? examples/s]

Dataset({
    features: ['Medicine Name', 'Composition', 'Uses', 'Side_effects', 'Image URL', 'Manufacturer', 'Excellent Review %', 'Average Review %', 'Poor Review %', 'text', 'input_ids', 'attention_mask'],
    num_rows: 11825
})

#**Training hyperparamters**
###**Data Collators**
Language modeling data collator comes in two modes:
* MLM
* CLM

#####**MLM**
MLM data collator: this is for masked language modeling, where we mask 15% of tokens and the model predict them. Default MLM is true.
#####**CLM**
CLM data collator: this is for causal language modeling where we mask all tokens to the right side of the current token, and expect the model to predict the next token at each step. Defult CLM is true.

In [15]:
training_arguments = TrainingArguments(
        output_dir = "output",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 2,
        learning_rate = 2e-4,
        lr_scheduler_type = "cosine",
        #EvaluationStrategy = "steps",
        save_strategy = "epoch",
        logging_steps = 30,
        max_steps = -1,
        num_train_epochs = 5,
    )
trainer = Trainer(
    model = model,
    train_dataset = tokenized_training_data["input_ids"],
    args = training_arguments,
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
)

#**Training**

In [16]:
trainer.train()
trainer.model.save_pretrained(new_model)
model.config.use_cache = True
model.eval()

Step,Training Loss
30,1.298400
60,0.673000
90,0.578200
120,0.516100
150,0.484400
180,0.501000
210,0.507800
240,0.432400
270,0.446200
300,0.430500


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): PhiForCausalLM(
      (transformer): PhiModel(
        (embd): Embedding(
          (wte): Embedding(51200, 2048)
          (drop): Dropout(p=0.0, inplace=False)
        )
        (h): ModuleList(
          (0-23): 24 x ParallelBlock(
            (ln): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (mixer): MHA(
              (rotary_emb): RotaryEmbedding()
              (Wqkv): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=6144, bias=False)
                )
                (lora_embedding_A): ParameterDict()
             

#**Testing the trained model**

In [35]:
def phi_stream(prompt):
  runtimeFlag = "cuda:0"
  inputs = tokenizer(f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{prompt}.\n\n### Response:\n ''', return_tensors="pt", return_attention_mask=False).to(runtimeFlag)
  streamer = TextStreamer(tokenizer, skip_prompt= True)
  _ = model.generate(**inputs, streamer=streamer, max_new_tokens=20)
phi_stream("What is the use of Avastin 400mg?")


Uses: Avastin 400mg is commonly used to treat  Breast cancer.



#**Merging the adpater and pretrained model**

In [18]:
base_model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map= {"": 0},
)
model = PeftModel.from_pretrained(base_model,new_model)
model = model.merge_and_unload()
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

The repository for microsoft/phi-1_5 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/phi-1_5.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


#**Push the model on Huggingface**

In [28]:
# !huggingface-cli login
model.push_to_hub("phi-1.5b-bf16-finetuned-medicine-conversational", commit_message="update", use_temp_dir = False)
tokenizer.push_to_hub("phi-1.5b-bf16-finetuned-medicine-conversational", commit_message="update",use_temp_dir = False)

model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/AamirAli123/phi-1.5b-bf16-finetuned-medicine-conversational/commit/ce471e4aa069ee728f65683865733f15743070e4', commit_message='update', commit_description='', oid='ce471e4aa069ee728f65683865733f15743070e4', pr_url=None, pr_revision=None, pr_num=None)

#**Inference with Local Model**

In [38]:
# Loading PEFT model
PEFT_MODEL = new_model
config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map= {"": 0},
)
peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)
peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

The repository for microsoft/phi-1_5 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/phi-1_5.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [52]:
# Function to generate responses from PEFT model.
def generate_answer(query):
  system_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.
  If the question is too complex, respond 'Kindly, consult a doctor for further queries.\n"""
  user_prompt = f"""### Instruction:\n{query}\n\n### Response: \n"""
  final_prompt = system_prompt + "\n" + user_prompt
  #device = "cuda:0"
  dashline = "-".join("" for i in range(50))
  # -----------------------------
  # encoding = tokenizer(final_prompt, return_tensors="pt").to(device)
  # outputs = model.generate(input_ids=encoding.input_ids, generation_config=GenerationConfig(max_new_tokens=256, pad_token_id = tokenizer.eos_token_id,
  #                                                                                                                    eos_token_id = tokenizer.eos_token_id, attention_mask = encoding.attention_mask, temperature=0.4, top_p=0.6, repetition_penalty=1.3, num_return_sequences=1))
  # text_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
  # print(dashline)
  # print(f'ORIGINAL MODEL RESPONSE:\n{text_output}')
  # ------------------------------
  print(dashline)
  peft_encoding = peft_tokenizer(final_prompt, return_tensors="pt")
  peft_outputs = peft_model.generate(input_ids = peft_encoding.input_ids, generation_config = GenerationConfig(max_new_tokens = 256, pad_token_id = peft_tokenizer.eos_token_id,
                                                                                                                     eos_token_id = peft_tokenizer.eos_token_id, attention_mask = peft_encoding.attention_mask, temperature=0.4,
                                                                                                               top_p=0.6, repetition_penalty=1.3, num_return_sequences=1))
  peft_text_output = peft_tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

  print(f'PEFT MODEL RESPONSE:\n{peft_text_output}')
  print(dashline)

In [47]:
# generate_answer("What is the use of Avastin 400mg?")
generate_answer("What is the use of Azithral 500 Tablet?")

-------------------------------------------------
PEFT MODEL RESPONSE:
Below is an instruction that describes a task. Write a response that appropriately completes the request.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.
  If the question is too complex, respond 'Kindly, consult a doctor for further queries.

### Instruction:
What is the use of Azithral 500 Tablet?

### Response: 
Azitheral 100mg Suspension Orange Sugar Free! It's commonly used to treat Treatment of Bacterial infections.


-------------------------------------------------


#**Inferenece with Huggingface Model**

In [48]:
# Loading PEFT model
peft_model = AutoModelForCausalLM.from_pretrained("AamirAli123/phi-1.5b-bf16-finetuned-medicine-conversational")
peft_tokenizer = AutoTokenizer.from_pretrained("AamirAli123/phi-1.5b-bf16-finetuned-medicine-conversational")

config.json:   0%|          | 0.00/765 [00:00<?, ?B/s]

The repository for AamirAli123/phi-1.5b-bf16-finetuned-medicine-conversational contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/AamirAli123/phi-1.5b-bf16-finetuned-medicine-conversational.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


model.safetensors:   0%|          | 0.00/2.84G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.37k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/131 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [53]:
# generate_answer("What is the use of Avastin 400mg?")
generate_answer("What is the use of Azithral 500 Tablet?")

-------------------------------------------------
PEFT MODEL RESPONSE:
Below is an instruction that describes a task. Write a response that appropriately completes the request.
  If you don't know the answer, respond 'Sorry, I don't know the answer to this question.
  If the question is too complex, respond 'Kindly, consult a doctor for further queries.

### Instruction:
What is the use of Azithral 500 Tablet?

### Response: 
Azitheral 100mg Suspension Orange Sugar Free! It's commonly used to treat Treatment of Bacterial infections.


-------------------------------------------------


#**streaming inference**

In [56]:
def phi_stream(prompt):
  #runtimeFlag = "cuda:0"
  inputs = peft_tokenizer(f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{prompt}.\n\n### Response:\n ''', return_tensors="pt", return_attention_mask=False)
  streamer = TextStreamer(tokenizer, skip_prompt= True)
  _ = peft_model.generate(**inputs, streamer=streamer, max_new_tokens=20)
phi_stream("What is the use of Avastin 400mg?")


Uses: Avastin 400mg is commonly used to treat  Breast cancer.

